# Analysing the customer Behaviour

Customer behavior here means the customers habit of buying food and drink. Is the any bias twards buying juice along with food. i.e., does he/she like the juice along with the food only?

- How many juice was sold before opening food section (before evening) and its percentage.
- How many juice went along with food after opening food section.
- How many juice went along with shawarma & its percentage.

##Importing necessary packages and mounting drive and authorizing

In [ ]:
## mounting drive
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [ ]:
## authenticating drive to get file names
from google.colab import auth
auth.authenticate_user()

In [ ]:
## Importing gspread to read google sheets

import gspread
from google.auth import default

creds,_ = default()
gc = gspread.authorize(creds)

In [ ]:
import subprocess as sp
import pandas as pd
import numpy as np

## Getting file names

In [ ]:
months = ['November','December','January']
files = []
for month in months:
  files = files + sp.getoutput(f'ls "/content/gdrive/MyDrive/Avocado Analysis/Data sheets/{month}"').split('\n')
files = [file.split('.')[0] for file in files]  ## removing the .gsheet extension
print("There are ",len(files),"files")
print(files)

There are  84 files
['10_11', '11_11', '1_11', '12_11', '13_11', '14_11', '15_11', '16_11', '17_11', '18_11', '19_11', '20_11', '21_11', '2_11', '22_11', '23_11', '24_11', '25_11', '26_11', '27_11', '28_11', '29_11', '30_11', '3_11', '4_11', '5_11', '6_11', '7_11', '8_11', '9_11', '10_12', '11_12', '1_12', '16_12', '17_12', '18_12', '19_12', '20_12', '21_12', '2_12', '22_12', '23_12', '24_12', '25_12', '26_12', '29_12', '31_12', '3_12', '4_12', '6_12', '7_12', '8_12', '9_12', '10_1', '11_1', '1_1', '12_1', '13_1', '14_1', '15_1', '16_1', '17_1', '18_1', '19_1', '20_1', '21_1', '2_1', '22_1', '23_1', '24_1', '25_1', '26_1', '27_1', '28_1', '29_1', '30_1', '31_1', '3_1', '4_1', '5_1', '6_1', '7_1', '8_1', '9_1']


## Extracting Behaviour pattern

In [ ]:
## finding the starting point(row) of food section
def findFoodStart(lst):
  for i in range(len(lst)):
    if lst[i]>0:
      return i

## Function for extracting the behaviour within a day
def behaviourInADay(date):
  if date in ['1_11','2_11','23_11','25_11','2_12','8_12','9_12','21_12']: ## Outliers shop only partially open
    return None
  try:
    beh={}
    beh['Date'] = date
    data = pd.DataFrame(gc.open(date).sheet1.get_all_records())  ## returns pandass DataFrame
    data.drop('-\n',axis = 'columns',inplace=True)
    for i in [0,1,2,3,4,6,8,10,12]:   ## type converting columns to int for ease of calculations
      data.iloc[:,i] = pd.to_numeric(data.iloc[:,i])

    data['Drinks_sum'] = data.iloc[:,[4,6,8,10,12]].sum(axis=1)
    data['Food_sum'] = data.iloc[:,0:4].sum(axis=1)

    data = data[data.iloc[:,[14,15]].sum(axis = 1) > 0] ## removing empty columns

    data['Drinks_only'] = np.where(data.Drinks_sum>data.Food_sum,data.Drinks_sum-data.Food_sum,0)
    data['DrinkWithFood'] = data.Drinks_sum - data.Drinks_only

    food_start  = findFoodStart(list(data.Food_sum))

    beh['Drinks_FN'] = sum(list(data.Drinks_sum)[:food_start])
    beh['Drinks_only'] = sum(list(data.Drinks_only)[food_start:])
    beh['DrinksWithFood'] = sum(list(data.DrinkWithFood)[food_start:])
  except Exception as e:
    print(date," Error : ",e)
    return None
  return beh



In [ ]:

behaviour_data = pd.DataFrame(columns = ['Date','Drinks_FN','Drinks_only','DrinksWithFood'])
for month in ['_11','_12','_1']:
  for day in range(1,32):
    if month == '_1' and day >19:
      break
    date = str(day)+month
    if date in files:
      new_row = behaviourInADay(date)
      if new_row != None:
        behaviour_data = behaviour_data.append(new_row,ignore_index =True)
print(behaviour_data.shape)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.


3_1  Error :  Unable to parse string "i" at position 50
(63, 4)


## Saving the behaviour pattern in csv file

In [ ]:
## Saving the aggregate data  to drive
path =  "/content/gdrive/MyDrive/Avocado Analysis/Calculations/Behaviour.csv"
with open(path, 'w', encoding = 'utf-8-sig') as f:
  behaviour_data.to_csv(f)